In [36]:
from bioexplorer import BioExplorer, Protein, Volume, Vector3, Quaternion

URL = 'localhost:5000'
be = BioExplorer(URL)
core = be.core_api()
be.reset_scene()

{'contents': 'Removed 0 models', 'status': True}

In [37]:
load_neuron = True
neuron_guid = 989
load_astrocyte = False
load_vasculature = False
use_sdf = True
generate_movie = False

scale = Vector3(100.0, 100.0, 100.0)
scene_center = [552.193724,1736.099481,235.409219]

In [38]:
if load_neuron:
    print('Loading neuron')
    neuron_sql_filter = 'guid=%d' % neuron_guid
    neuron_assembly_name = 'Neuron'
    be.remove_assembly(neuron_assembly_name)
    be.add_assembly(neuron_assembly_name)
    be.add_neurons(
        assembly_name=neuron_assembly_name,
        morphology_color_scheme=be.MORPHOLOGY_COLOR_SCHEME_SECTION,
        population_name='o1',
        use_sdf=use_sdf,
        load_somas=True, load_basal_dendrites=True, load_apical_dendrites=True, load_axon=True,
        load_synapses=True,
        generate_internals=False, generate_externals=False,
        sql_node_filter=neuron_sql_filter, scale=scale)

if load_astrocyte:
    print('Loading astrocytes...')
    astrocytes_sql_filter='guid=7178'
    assembly_name = 'Astrocytes'
    be.remove_assembly(assembly_name)
    vasculature_assembly = be.add_assembly(assembly_name)
    vasculature_model = be.add_astrocytes(
        assembly_name=assembly_name,
        population_name='ngv',
        radius_multiplier=1.0,
        use_sdf=use_sdf, generate_internals=True,
        load_somas=True, load_dendrites=True, load_end_feet=True,
        sql_filter=astrocytes_sql_filter, scale=scale)

if load_vasculature:
    vasculature_sql_filter = 'sqrt(pow(x - %f, 2) + pow(y - %f, 2) + pow(z - %f, 2)) < 100' % (scene_center[0], scene_center[1], scene_center[2])
    print('Loading vasculature...')
    assembly_name = 'Vasculature'
    population_name = 'Vasculature'
    be.remove_assembly(assembly_name)
    vasculature_assembly = be.add_assembly(assembly_name)

    vasculature_model = be.add_vasculature(
        assembly_name=assembly_name,
        population_name=population_name,
        quality=be.GEOMETRY_QUALITY_MEDIUM,
        use_sdf=use_sdf,
        radius_multiplier=0.9,
        sql_filter=vasculature_sql_filter, scale=scale)

Loading neuron


In [39]:
def set_neuron_materials(model_id, opacity, color, shading_mode):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    reflection_indices = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    for material_id in material_ids:
        mid = material_id % be.NB_MATERIALS_PER_MORPHOLOGY
        if mid == be.NEURON_MATERIAL_MITOCHONDRION:
            opacities.append(1.0)
            # opacities.append(0.25)
            colors.append([0.4, 0.08, 0.4])
            shading_modes.append(be.SHADING_MODE_NONE)
            # shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.5)
            user_params.append(1.0 / scale.x)
            specular_exponents.append(6.0)
            emissions.append(0.5)
            reflection_indices.append(0.0)
        elif mid == be.NEURON_MATERIAL_NUCLEUS:
            opacities.append(1.0)
            colors.append([0.8, 0.9, 0.3])
            shading_modes.append(be.SHADING_MODE_NONE)
            glossinesses.append(1.0)
            user_params.append(0.2 / scale.x)
            specular_exponents.append(30.0)
            emissions.append(0.0)
            reflection_indices.append(0.0)
        elif mid == be.NEURON_MATERIAL_SOMA or mid==be.NEURON_MATERIAL_AXON:
            opacities.append(opacity)
            colors.append(color)
            shading_modes.append(shading_mode)
            glossinesses.append(0.75)
            user_params.append(0.05 / scale.x)
            specular_exponents.append(5.0)
            emissions.append(0.0)
            reflection_indices.append(0.0)
        elif mid == be.NEURON_MATERIAL_MYELIN_STEATH:
            # opacities.append(0.3)
            opacities.append(1.0)
            colors.append([0.59, 0.82, 0.14])
            shading_modes.append(be.SHADING_MODE_PERLIN)
            glossinesses.append(0.1)
            user_params.append(0.1 / scale.x)
            specular_exponents.append(5.0)
            emissions.append(0.0)
            reflection_indices.append(0.0)
        else:
            # Membrane
            opacities.append(opacity)
            colors.append(color)
            shading_modes.append(be.SHADING_MODE_NONE)
            glossinesses.append(0.7)
            user_params.append(0.02 / scale.x)
            specular_exponents.append(5.0)
            emissions.append(0.0)
            reflection_indices.append(0.0)
            
        refraction_indices.append(1.2)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        reflection_indices=reflection_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions
    )

def set_astrocyte_materials(model_id, opacity, color, shading_mode):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    reflection_indices = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    for material_id in material_ids:
        mid = material_id % be.NB_MATERIALS_PER_MORPHOLOGY
        opacities.append(opacity)
        colors.append(color)
        shading_modes.append(shading_mode)
        glossinesses.append(0.1)
        user_params.append(0.01 / scale.x)
        specular_exponents.append(5.0)
        emissions.append(0.0)
        reflection_indices.append(0.0)
        refraction_indices.append(1.01)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        reflection_indices=reflection_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions
    )

def set_vasculature_materials(model_id):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    
    material_ids = be.get_material_ids(model_id)['ids']
    for _ in material_ids:
        opacities.append(0.5)
        colors.append([1, 0, 0])
        shading_modes.append(be.SHADING_MODE_NONE)
        # shading_modes.append(be.SHADING_MODE_PERLIN)
        glossinesses.append(0.9)
        user_params.append(0.01 / scale.x)
        specular_exponents.append(3.0)
        refraction_indices.append(2.5)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses
    )

model_ids = be.get_model_ids()['ids']
index = 0
if load_neuron:
    set_neuron_materials(
        model_ids[index], 0.6, [206 / 256, 185 / 256, 135 / 256],
        be.SHADING_MODE_NONE)
    index += 1
if load_astrocyte:
    set_astrocyte_materials(
        model_ids[index], 1.0, [206 / 256, 185 / 256, 135 / 256],
        be.SHADING_MODE_NONE)
    index += 1
if load_vasculature:
    set_vasculature_materials(model_ids[index])
status = core.set_renderer()

In [40]:
status = be.reset_camera()
status = core.set_renderer()

## Step 1 - From nucleus to bouton

In [41]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)

resource_folder = '../../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'
membrane_folder = pdb_folder + 'membrane/'
lipids_folder = membrane_folder + 'lipids/'
transporters_folder = pdb_folder + 'transporters/'
tubules_folder = pdb_folder + 'tubules/'

points = be.get_neuron_section_points(
    assembly_name=neuron_assembly_name, neuron_guid=neuron_guid, section_guid=0)


### Microtubules

In [43]:
import random
import numpy as np
import math

nb_microtubules = len(points) - 1

for i in range(nb_microtubules):

    # be.add_sphere('sphere', Vector3(p0[0], p0[1], p0[2]), 10.0, Vector3(1,1,1))

    p0 = points[i]
    p1 = points[i+1]

    src = np.array(p0[:3])
    dst = np.array(p1[:3])
    forwardVector = np.subtract(dst, src)
    forward_length = np.sqrt(np.sum(forwardVector**2))
    microtubule_size  = Vector3(5.0, forward_length, 0.0)

    q = be.look_at(
        Vector3(p0[0], p0[1], p0[2]),
        Vector3(p1[0], p1[1], p1[2])
    )
    # print('x=%f, y=%f, z=%f, w=%f' % (q.x, q.y, q.z, q.w))

    # position = Vector3(
    #     p0[0] + random.randint(int(-scale.x / 1.0), int(scale.x / 1.0)),
    #     p0[1] + random.randint(int(-scale.y / 1.0), int(scale.y / 1.0)),
    #     p0[2] + random.randint(int(-scale.z / 1.0), int(scale.z / 1.0))
    # )

    position = Vector3(p0[0], p0[1], p0[2])
    
    name = 'Microtubule%05d' % i

    # ACE2 receptor definition
    tubule = Protein(
        name=name + '_TUB',
        source=tubules_folder + '1tub.pdb', 
        rotation=Quaternion(0.0, 0.0, 0.707, 0.707),
        occurences=microtubule_size.y * 5.0)

    volume = Volume(
        name=name,
        shape=be.ASSEMBLY_SHAPE_HELIX, shape_params=microtubule_size,
        protein=tubule)

    status = be.add_volume(
        position=position,
        # rotation=Quaternion(q.z, q.w, q.x, q.y),
        # rotation=Quaternion( 0.318, 0.239, 0.733, -0.552), # w, z, x, y 
        rotation=q,
        volume=volume)

    status = be.set_protein_color_scheme(
        assembly_name=name,
        name=name + '_TUB',
        color_scheme=be.COLOR_SCHEME_CHAINS,
        palette_name='Set2', palette_size=5)
core.set_renderer(subsampling=4, max_accum_frames=1)

True

## Movie

### Inside Axon, follow microtubule

In [44]:
import math

nb_frames = 120
aperture = 0.01
focus = 2.938
key_frames = list()

for i in range(0, len(points)-1, 5):
    origin = [
        points[i][0],
        points[i][1] + 50,
        points[i][2]
    ]
    
    target = [
        points[i+1][0],
        points[i+1][1],
        points[i+1][2]
    ]

    dir = [0.0, 0.0, 0.0]
    ldir = 0.0
    for k in range(3):
        l = target[k] - origin[k]
        dir[k] = l * scale.to_list()[k]
        ldir += l * l
    for k in range(3):
        dir[k] /= math.sqrt(ldir)
    
    mm.set_camera(origin=origin, direction=dir, up=[0, 1, 0])
    key_frames.append(mm.get_camera())
    # core.set_renderer()

mm.build_camera_path(key_frames, nb_frames, nb_frames)

In [45]:
for i in range(mm.get_nb_frames()):
    mm.set_current_frame(i)
    import time
    time.sleep(0.1)
    core.set_renderer()

KeyboardInterrupt: 

In [ ]:
stop

In [ ]:
status = core.set_renderer(current='bio_explorer',
    subsampling=4, max_accum_frames=64)
params = core.BioExplorerRendererParams()
params.exposure = 1.2
params.max_bounces = 10
params.epsilon_factor = 500.0
params.fog_start = 40.0
params.fog_thickness = 250.0
params.gi_samples = 1
params.gi_weight = 0.2
params.gi_distance = 3.0
params.shadows = 1.0
params.soft_shadows = 1.0
status = core.set_renderer_params(params)

In [ ]:
if generate_movie:
    for frame in range(mm.get_nb_frames()):
        mm.set_current_frame(frame)
        mm.create_snapshot(
            renderer='circuit_explorer_advanced',
            size=[960,540], samples_per_pixel=32,
            path='/gpfs/bbp.cscs.ch/project/proj129/movies/neuromodulation/perspective/960x540/circuit_explorer_advanced',
            base_name='%05d' % frame
        )
else:
    import time
    for frame in range(0, mm.get_nb_frames(), 2):
        mm.set_current_frame(frame)
        time.sleep(0.2)